In [35]:
import numpy as np
from qinterp import TricubicScalarInterpolator
from scipy.interpolate import RegularGridInterpolator
from pathlib import Path

In [16]:
tests_dir = Path("..") / "tests"
data_path = tests_dir / "fields" / "Example3DScalarField.csv"
interp_data_path = tests_dir / "fields" / "Example3DScalarFieldResults.npz"

In [20]:
data = np.genfromtxt(data_path, delimiter=',')
tri = TricubicScalarInterpolator(data)

A = np.load(interp_data_path)
coords = A['coords']
xi = coords[3]

field = tri.field(xi)
grad = tri.gradient(xi)

assert np.allclose(field, A['single_field'])
assert np.allclose(grad, A['single_grad'])

fields = tri.field(coords)
grads = tri.gradient(coords)

assert np.allclose(fields, A['multi_field'])
assert np.allclose(grads, A['multi_grad'])
assert np.allclose(tri(coords[3], d=2), tri.hessian(coords)[3])


In [23]:
coords.shape, tri.field(coords).shape

((20, 3), (20, 1))

In [31]:
def shapes(x):
    return [x.shape] + [tri(x, d=d).shape for d in range(3)]

shapes(coords)

[(20, 3), (20, 1), (20, 3), (20, 3, 3)]

In [32]:
shapes(coords[3])

[(3,), (), (3,), (3, 3)]

In [33]:
shapes(coords[3].reshape(1, -1))

[(1, 3), (1, 1), (1, 3), (1, 3, 3)]

In [36]:
points = tri.x, tri.y, tri.z
values = tri.field_data

grid_interp = RegularGridInterpolator(points, values, method="cubic")


In [37]:
[coords.shape, grid_interp(coords).shape]

[(20, 3), (20,)]

In [39]:
xi = coords[3]
[xi.shape, grid_interp(xi).shape]

[(3,), (1,)]

In [40]:
xi = coords[3].reshape(1, -1)
[xi.shape, grid_interp(xi).shape]

[(1, 3), (1,)]

In [47]:
%%timeit
tri(coords[3], d=0)

79.2 µs ± 4.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [48]:
%%timeit
tri(coords[3].reshape(1, -1), d=0)

270 µs ± 6.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [50]:
%%timeit
tri(coords)

315 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [49]:
%%timeit
grid_interp(coords)

19.5 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
